# 패스트텍스트(FastText)

Word2Vec 이후에 나온 것으로, Word2Vec은 단어를 쪼갤 수 없는 단위로 생각한다면 FastText는 하나의 단어 안에도 내부 단어(subword)가 있다고 고려한다.

## Subword

FastText에서는 각 단어를 n-gram의 구성이라고 취급한다. 따라서 n 값에 따라 subword가 토큰으로 만들어진다. 예를 들어, apple이라는 단어의 경우, n=3일 때 subword는 다음과 같이 지정된다. app, ppl, ple, \<ap, le\>, \<apple\>. 이 때, 내부 단어의 벡터값은 설정한 모든 n들에서의 벡터 합이다.

이후에 단어의 벡터값은 내부 단어들의 벡터 합으로 구성한다.

# Word2Vec과 비교하기 위한 코드

In [1]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

In [2]:
# 훈련 데이터(XML)
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x1357a6400>)

In [3]:
# 전처리
with open("ted_en-20160408.xml") as targetXML:
    target_text = etree.parse(targetXML)
    parse_text = "\n".join(target_text.xpath("//content/text()")) # <content> 태그 안의 내용만 가져오기
    content_text = re.sub(r"\([^)]*\)", "", parse_text) # (Audio), (Laughter) 등의 배경음 제거
    sent_text = sent_tokenize(content_text)

    normalized_text = []
    for string in sent_text:
        tokens = re.sub(r"[^a-z0-9]+", " ", string.lower()) # 구두점 제거 및 소문자 변환
        normalized_text.append(tokens)

    res =[word_tokenize(sentence) for sentence in normalized_text]

print(len(res))

273424


In [4]:
print(res[:3])

[['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new'], ['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation'], ['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']]


In [5]:
# 학습
from gensim.models import Word2Vec, KeyedVectors

# vector_size: 임베딩 벡터의 차원, window: 윈도우 크기, min_count: 단어 최소 빈도 수, workers: 프로세스 수, sg: 0 = CBOW, 1 = Skip-gram
model = Word2Vec(sentences=res, vector_size=100, window=5, min_count=5, workers=4, sg=0)

In [6]:
model_res = model.wv.most_similar("man") # 토큰 man과 가장 유사한 단어들

model_res

[('woman', 0.8494877219200134),
 ('guy', 0.8141617774963379),
 ('boy', 0.7771295309066772),
 ('lady', 0.7554702758789062),
 ('gentleman', 0.7519720196723938),
 ('soldier', 0.7374513149261475),
 ('girl', 0.7356285452842712),
 ('kid', 0.7040250301361084),
 ('friend', 0.6623315811157227),
 ('poet', 0.6444184184074402)]

In [7]:
# 저장 및 불러오기
model.wv.save_word2vec_format("eng_w2v")
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v")

loaded_model.most_similar("electrofishing")

KeyError: "Key 'electrofishing' not present in vocabulary"

## FastText 학습 코드

In [9]:
from gensim.models import FastText

model = FastText(res, vector_size=100, window=5, min_count=5, workers=4, sg=1)

In [10]:
model.wv.most_similar("electrofishing")

[('electrolux', 0.8648669123649597),
 ('electrolyte', 0.8615568280220032),
 ('electro', 0.8566197752952576),
 ('electroencephalogram', 0.8541638851165771),
 ('electroshock', 0.8430773019790649),
 ('electrogram', 0.8309141993522644),
 ('airbag', 0.8303189873695374),
 ('electron', 0.824195921421051),
 ('airbus', 0.8194237351417542),
 ('electromagnet', 0.8174201846122742)]